In [1]:
!ls -lash output_michaelrosen2

total 18G
4.0K drwxrwxr-x  2 ubuntu ubuntu 4.0K Feb 14 19:00 .
4.0K drwxrwxr-x 23 ubuntu ubuntu 4.0K Feb 15 20:08 ..
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 11:43 checkpoint_Tacotron2_1000.pt
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 12:02 checkpoint_Tacotron2_1020.pt
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 12:21 checkpoint_Tacotron2_1040.pt
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 12:40 checkpoint_Tacotron2_1060.pt
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 12:58 checkpoint_Tacotron2_1080.pt
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 13:17 checkpoint_Tacotron2_1100.pt
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 13:36 checkpoint_Tacotron2_1120.pt
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 13:55 checkpoint_Tacotron2_1140.pt
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 14:14 checkpoint_Tacotron2_1160.pt
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 14:33 checkpoint_Tacotron2_1180.pt
323M -rw-rw-r--  1 ubuntu ubuntu 323M Feb 14 14:52 checkpoint_Tacotron2_1200.pt
323

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
model_config = dict(
    mask_padding=True,
    n_mel_channels=80,
    n_symbols=148,
    symbols_embedding_dim=512,
    #encoder
    encoder_kernel_size=5,
    encoder_n_convolutions=3,
    encoder_embedding_dim=512,
    # attention
    attention_rnn_dim=1024,
    attention_dim=128,
    # attention location
    attention_location_n_filters=32,
    attention_location_kernel_size=31,
    # decoder
    n_frames_per_step=1,
    decoder_rnn_dim=1024,
    prenet_dim=256,
    max_decoder_steps=2000,
    gate_threshold=0.5,
    p_attention_dropout=0.1,
    p_decoder_dropout=0.1,
    # postnet
    postnet_embedding_dim=512,
    postnet_kernel_size=5,
    postnet_n_convolutions=5,
    decoder_no_early_stopping=False,
)

In [5]:
import argparse

import torch
from torch.utils.data import DataLoader
import numpy as np

from common.utils import load_filepaths_and_text
from tacotron2.data_function import TextMelLoader, TextMelCollate, batch_to_gpu
import loss_functions
from train import parse_args as train_parse_args

import models
from tacotron2.arg_parser import tacotron2_parser
from IPython.display import Audio
import os

In [6]:
def parse_args(parser):
    """
    Parse commandline arguments.
    """
    parser.add_argument('--text-cleaners', nargs='*',
                        default=['english_cleaners'], type=str,
                        help='Type of text cleaners for input text')
    parser.add_argument('--max-wav-value', default=32768.0, type=float,
                        help='Maximum audiowave value')
    parser.add_argument('--sampling-rate', default=22050, type=int,
                        help='Sampling rate')
    parser.add_argument('--filter-length', default=1024, type=int,
                        help='Filter length')
    parser.add_argument('--hop-length', default=256, type=int,
                        help='Hop (stride) length')
    parser.add_argument('--win-length', default=1024, type=int,
                        help='Window length')
    parser.add_argument('--mel-fmin', default=0.0, type=float,
                        help='Minimum mel frequency')
    parser.add_argument('--mel-fmax', default=8000.0, type=float,
                        help='Maximum mel frequency')
    parser.add_argument('--n-mel-channels', default=80, type=int,
                        help='Number of bins in mel-spectrograms')

    return parser

parser = argparse.ArgumentParser(description="Create mels with teacher forcing")
parser = parse_args(parser)
#parser = train_parse_args(parser)
args = parser.parse_args([
    #"--output", "output", "--model-name", "Tacotron2", 
    #"--epochs", "100", "--learning-rate", "1e-4",
    #"-bs", "10"
])


In [7]:
model = models.get_model("Tacotron2", model_config, cpu_run=False)
# pretrained = torch.load("./output_michaelrosen2/checkpoint_Tacotron2_last.pt")
pretrained = torch.load("./output_trebek/checkpoint_Tacotron2_last.pt")

In [8]:
model.load_state_dict(pretrained["state_dict"])
model.cuda()
model.eval()
print('model ready')

model ready


In [9]:
# dataset_path = "/home/ubuntu/michael-rosen/"
# audiopaths_and_text = "/home/ubuntu/michael-rosen/michaelrosen_only_arpa_audio_text_train_filelist.txt"
# melpaths_and_text = "/home/ubuntu/michael-rosen/michaelrosen_only_arpa_mel_text_train_filelist.txt"
dataset_path = "/home/ubuntu/trebek/"
audiopaths_and_text = "/home/ubuntu/trebek/trebek_only_arpa_audio_text_train_filelist.txt"
melpaths_and_text = "/home/ubuntu/trebek/trebek_only_arpa_mel_text_train_filelist.txt"
melpaths_and_text_list = load_filepaths_and_text(
    dataset_path, 
    melpaths_and_text,
)
audiopaths_and_text_list = load_filepaths_and_text(
    dataset_path, 
    audiopaths_and_text
)
args.load_mel_from_disk = True
mel_loader = TextMelLoader(dataset_path, melpaths_and_text, args)
data_loader = DataLoader(mel_loader, collate_fn=TextMelCollate(1), batch_size=2, shuffle=False)


In [10]:
idx = 0
model.eval()
tf_dir = "/home/ubuntu/hg_tune_data/mels"
with torch.no_grad():
    for i, batch in enumerate(data_loader):
        x, y, num_items = batch_to_gpu(batch)
        out = model(x)
        mel_out, mel_out_postnet, gate_outs, alignments = out
        
        lengths = np.array([len(mel_loader[i][0]) for i in range(idx, idx + mel_out.size(0))])
        apt = np.array([mel_loader.audiopaths_and_text[i] for i in range(idx, idx + mel_out.size(0))])
        apt = apt[np.argsort(lengths)[::-1]]
        
        output_lengths = batch[4]
        for i, el in enumerate(apt):
            filename = el[0].split("/")[-1].replace(".pt", ".npy")
            print(filename) # filename
            output_length = int(output_lengths[i].item())
            mel = mel_out_postnet[i, :, :output_length].unsqueeze(0).data.cpu().numpy()
            print(mel.shape)
            np.save(os.path.join(tf_dir, filename), mel)
        idx += mel_out.size(0)

trebek_1346.npy
(1, 80, 809)
trebek_543.npy
(1, 80, 526)
trebek_2390.npy
(1, 80, 386)
trebek_1002.npy
(1, 80, 365)
trebek_1695.npy
(1, 80, 782)
trebek_1125.npy
(1, 80, 688)
trebek_1376.npy
(1, 80, 683)
trebek_1618.npy
(1, 80, 391)
trebek_1773.npy
(1, 80, 619)
trebek_1594.npy
(1, 80, 321)
trebek_2116.npy
(1, 80, 597)
trebek_2507.npy
(1, 80, 331)
trebek_2097.npy
(1, 80, 686)
trebek_658.npy
(1, 80, 692)
trebek_662.npy
(1, 80, 714)
trebek_600.npy
(1, 80, 510)
trebek_2324.npy
(1, 80, 672)
trebek_1772.npy
(1, 80, 321)
trebek_2545.npy
(1, 80, 401)
trebek_2553.npy
(1, 80, 313)
trebek_1467.npy
(1, 80, 789)
trebek_1204.npy
(1, 80, 523)
trebek_1756.npy
(1, 80, 469)
trebek_2546.npy
(1, 80, 241)
trebek_76.npy
(1, 80, 727)
trebek_886.npy
(1, 80, 628)
trebek_684.npy
(1, 80, 792)
trebek_292.npy
(1, 80, 554)
trebek_269.npy
(1, 80, 900)
trebek_1461.npy
(1, 80, 579)
trebek_961.npy
(1, 80, 644)
trebek_1607.npy
(1, 80, 466)
trebek_259.npy
(1, 80, 497)
trebek_1871.npy
(1, 80, 280)
trebek_2401.npy
(1, 80, 59

trebek_433.npy
(1, 80, 825)
trebek_2463.npy
(1, 80, 272)
trebek_1185.npy
(1, 80, 709)
trebek_1066.npy
(1, 80, 618)
trebek_719.npy
(1, 80, 637)
trebek_2385.npy
(1, 80, 422)
trebek_709.npy
(1, 80, 380)
trebek_2560.npy
(1, 80, 318)
trebek_341.npy
(1, 80, 645)
trebek_1493.npy
(1, 80, 471)
trebek_607.npy
(1, 80, 670)
trebek_1043.npy
(1, 80, 507)
trebek_518.npy
(1, 80, 691)
trebek_2302.npy
(1, 80, 380)
trebek_39.npy
(1, 80, 1036)
trebek_1808.npy
(1, 80, 298)
trebek_1234.npy
(1, 80, 745)
trebek_2486.npy
(1, 80, 471)
trebek_71.npy
(1, 80, 920)
trebek_635.npy
(1, 80, 750)
trebek_1212.npy
(1, 80, 526)
trebek_1041.npy
(1, 80, 440)
trebek_327.npy
(1, 80, 786)
trebek_2314.npy
(1, 80, 597)
trebek_1516.npy
(1, 80, 672)
trebek_1799.npy
(1, 80, 504)
trebek_1176.npy
(1, 80, 668)
trebek_1833.npy
(1, 80, 510)
trebek_2326.npy
(1, 80, 713)
trebek_270.npy
(1, 80, 432)
trebek_720.npy
(1, 80, 796)
trebek_338.npy
(1, 80, 509)
trebek_213.npy
(1, 80, 694)
trebek_125.npy
(1, 80, 639)
trebek_367.npy
(1, 80, 650)
tr

trebek_668.npy
(1, 80, 755)
trebek_1994.npy
(1, 80, 554)
trebek_1222.npy
(1, 80, 711)
trebek_793.npy
(1, 80, 572)
trebek_1444.npy
(1, 80, 728)
trebek_1475.npy
(1, 80, 654)
trebek_2078.npy
(1, 80, 603)
trebek_1403.npy
(1, 80, 473)
trebek_1092.npy
(1, 80, 601)
trebek_2497.npy
(1, 80, 409)
trebek_1116.npy
(1, 80, 572)
trebek_729.npy
(1, 80, 482)
trebek_1899.npy
(1, 80, 771)
trebek_2093.npy
(1, 80, 477)
trebek_268.npy
(1, 80, 810)
trebek_2086.npy
(1, 80, 567)
trebek_1578.npy
(1, 80, 577)
trebek_1044.npy
(1, 80, 432)
trebek_2338.npy
(1, 80, 690)
trebek_1986.npy
(1, 80, 450)
trebek_1330.npy
(1, 80, 471)
trebek_1308.npy
(1, 80, 340)
trebek_1969.npy
(1, 80, 639)
trebek_1474.npy
(1, 80, 533)
trebek_54.npy
(1, 80, 422)
trebek_1161.npy
(1, 80, 318)
trebek_889.npy
(1, 80, 729)
trebek_214.npy
(1, 80, 786)
trebek_931.npy
(1, 80, 608)
trebek_521.npy
(1, 80, 492)
trebek_1576.npy
(1, 80, 628)
trebek_1450.npy
(1, 80, 587)
trebek_1776.npy
(1, 80, 522)
trebek_979.npy
(1, 80, 499)
trebek_1268.npy
(1, 80, 5

trebek_2333.npy
(1, 80, 591)
trebek_2473.npy
(1, 80, 342)
trebek_1119.npy
(1, 80, 653)
trebek_1890.npy
(1, 80, 690)
trebek_1782.npy
(1, 80, 685)
trebek_2352.npy
(1, 80, 476)
trebek_1543.npy
(1, 80, 690)
trebek_1944.npy
(1, 80, 745)
trebek_1386.npy
(1, 80, 548)
trebek_1233.npy
(1, 80, 399)
trebek_1405.npy
(1, 80, 809)
trebek_2330.npy
(1, 80, 531)
trebek_805.npy
(1, 80, 603)
trebek_151.npy
(1, 80, 513)
trebek_771.npy
(1, 80, 719)
trebek_743.npy
(1, 80, 381)
trebek_2084.npy
(1, 80, 858)
trebek_2112.npy
(1, 80, 704)
trebek_701.npy
(1, 80, 931)
trebek_1069.npy
(1, 80, 564)
trebek_83.npy
(1, 80, 740)
trebek_1322.npy
(1, 80, 614)
trebek_746.npy
(1, 80, 623)
trebek_1656.npy
(1, 80, 569)
trebek_2551.npy
(1, 80, 546)
trebek_2188.npy
(1, 80, 671)
trebek_2092.npy
(1, 80, 468)
trebek_103.npy
(1, 80, 342)
trebek_1729.npy
(1, 80, 572)
trebek_2402.npy
(1, 80, 295)
trebek_1513.npy
(1, 80, 610)
trebek_2223.npy
(1, 80, 481)
trebek_2428.npy
(1, 80, 641)
trebek_744.npy
(1, 80, 628)
trebek_1383.npy
(1, 80, 

trebek_910.npy
(1, 80, 406)
trebek_1113.npy
(1, 80, 261)
trebek_1596.npy
(1, 80, 560)
trebek_1433.npy
(1, 80, 349)
trebek_1123.npy
(1, 80, 615)
trebek_1524.npy
(1, 80, 342)
trebek_1247.npy
(1, 80, 549)
trebek_627.npy
(1, 80, 541)
trebek_1592.npy
(1, 80, 681)
trebek_2165.npy
(1, 80, 641)
trebek_495.npy
(1, 80, 686)
trebek_1025.npy
(1, 80, 644)
trebek_340.npy
(1, 80, 584)
trebek_369.npy
(1, 80, 442)
trebek_1488.npy
(1, 80, 871)
trebek_722.npy
(1, 80, 598)
trebek_1622.npy
(1, 80, 530)
trebek_1863.npy
(1, 80, 466)
trebek_1214.npy
(1, 80, 479)
trebek_596.npy
(1, 80, 476)
trebek_2212.npy
(1, 80, 521)
trebek_603.npy
(1, 80, 414)
trebek_541.npy
(1, 80, 597)
trebek_2292.npy
(1, 80, 577)
trebek_800.npy
(1, 80, 427)
trebek_2535.npy
(1, 80, 352)
trebek_972.npy
(1, 80, 635)
trebek_841.npy
(1, 80, 525)
trebek_1491.npy
(1, 80, 794)
trebek_1203.npy
(1, 80, 556)
trebek_1849.npy
(1, 80, 632)
trebek_2423.npy
(1, 80, 610)
trebek_1822.npy
(1, 80, 603)
trebek_311.npy
(1, 80, 582)
trebek_6.npy
(1, 80, 900)
t

trebek_2417.npy
(1, 80, 603)
trebek_827.npy
(1, 80, 575)
trebek_1521.npy
(1, 80, 665)
trebek_70.npy
(1, 80, 727)
trebek_383.npy
(1, 80, 791)
trebek_1210.npy
(1, 80, 507)
trebek_33.npy
(1, 80, 846)
trebek_677.npy
(1, 80, 719)
trebek_1473.npy
(1, 80, 764)
trebek_2407.npy
(1, 80, 306)
trebek_406.npy
(1, 80, 305)
trebek_763.npy
(1, 80, 293)
trebek_1999.npy
(1, 80, 587)
trebek_1669.npy
(1, 80, 442)
trebek_954.npy
(1, 80, 668)
trebek_1780.npy
(1, 80, 598)
trebek_20.npy
(1, 80, 899)
trebek_713.npy
(1, 80, 654)
trebek_2438.npy
(1, 80, 783)
trebek_308.npy
(1, 80, 547)
trebek_465.npy
(1, 80, 691)
trebek_2278.npy
(1, 80, 388)
trebek_1663.npy
(1, 80, 752)
trebek_1088.npy
(1, 80, 742)
trebek_139.npy
(1, 80, 685)
trebek_1834.npy
(1, 80, 467)
trebek_1857.npy
(1, 80, 634)
trebek_679.npy
(1, 80, 367)
trebek_2154.npy
(1, 80, 751)
trebek_401.npy
(1, 80, 587)
trebek_161.npy
(1, 80, 758)
trebek_2471.npy
(1, 80, 541)
trebek_2204.npy
(1, 80, 684)
trebek_127.npy
(1, 80, 261)
trebek_1869.npy
(1, 80, 628)
trebe

trebek_150.npy
(1, 80, 711)
trebek_488.npy
(1, 80, 495)
trebek_1536.npy
(1, 80, 773)
trebek_1347.npy
(1, 80, 724)
trebek_1062.npy
(1, 80, 605)
trebek_180.npy
(1, 80, 479)
trebek_379.npy
(1, 80, 584)
trebek_1535.npy
(1, 80, 515)
trebek_1279.npy
(1, 80, 722)
trebek_2449.npy
(1, 80, 525)
trebek_1784.npy
(1, 80, 750)
trebek_967.npy
(1, 80, 657)
trebek_1901.npy
(1, 80, 699)
trebek_1178.npy
(1, 80, 502)
trebek_659.npy
(1, 80, 659)
trebek_2150.npy
(1, 80, 729)
trebek_2273.npy
(1, 80, 779)
trebek_1527.npy
(1, 80, 388)
trebek_1227.npy
(1, 80, 603)
trebek_1442.npy
(1, 80, 342)
trebek_1552.npy
(1, 80, 701)
trebek_2467.npy
(1, 80, 316)
trebek_2105.npy
(1, 80, 637)
trebek_404.npy
(1, 80, 616)
trebek_97.npy
(1, 80, 706)
trebek_1788.npy
(1, 80, 525)
trebek_283.npy
(1, 80, 517)
trebek_2301.npy
(1, 80, 349)
trebek_2130.npy
(1, 80, 730)
trebek_749.npy
(1, 80, 649)
trebek_1431.npy
(1, 80, 814)
trebek_944.npy
(1, 80, 680)
trebek_1224.npy
(1, 80, 597)
trebek_1964.npy
(1, 80, 432)
trebek_1510.npy
(1, 80, 53

trebek_572.npy
(1, 80, 649)
trebek_1737.npy
(1, 80, 486)
trebek_1837.npy
(1, 80, 615)
trebek_912.npy
(1, 80, 422)
trebek_575.npy
(1, 80, 647)
trebek_1406.npy
(1, 80, 494)
trebek_409.npy
(1, 80, 626)
trebek_2434.npy
(1, 80, 446)
trebek_2054.npy
(1, 80, 538)
trebek_1412.npy
(1, 80, 404)
trebek_2359.npy
(1, 80, 890)
trebek_1285.npy
(1, 80, 587)
trebek_1573.npy
(1, 80, 538)
trebek_1429.npy
(1, 80, 409)
trebek_876.npy
(1, 80, 670)
trebek_1952.npy
(1, 80, 339)
trebek_175.npy
(1, 80, 614)
trebek_959.npy
(1, 80, 507)
trebek_234.npy
(1, 80, 714)
trebek_1691.npy
(1, 80, 565)
trebek_1882.npy
(1, 80, 804)
trebek_92.npy
(1, 80, 425)
trebek_1980.npy
(1, 80, 569)
trebek_81.npy
(1, 80, 282)
trebek_2066.npy
(1, 80, 647)
trebek_403.npy
(1, 80, 541)
trebek_1778.npy
(1, 80, 752)
trebek_8.npy
(1, 80, 632)
trebek_1384.npy
(1, 80, 548)
trebek_1481.npy
(1, 80, 360)
trebek_2469.npy
(1, 80, 427)
trebek_291.npy
(1, 80, 393)
trebek_1879.npy
(1, 80, 678)
trebek_2193.npy
(1, 80, 613)
trebek_1605.npy
(1, 80, 505)
tr

trebek_2158.npy
(1, 80, 613)
trebek_1083.npy
(1, 80, 422)
trebek_836.npy
(1, 80, 649)
trebek_976.npy
(1, 80, 584)
trebek_1350.npy
(1, 80, 657)
trebek_2418.npy
(1, 80, 556)
trebek_1661.npy
(1, 80, 627)
trebek_133.npy
(1, 80, 484)
trebek_1223.npy
(1, 80, 815)
trebek_2441.npy
(1, 80, 525)
trebek_2030.npy
(1, 80, 634)
trebek_1762.npy
(1, 80, 553)
trebek_1774.npy
(1, 80, 892)
trebek_2489.npy
(1, 80, 461)
trebek_1226.npy
(1, 80, 596)
trebek_1868.npy
(1, 80, 709)
trebek_1237.npy
(1, 80, 601)
trebek_2138.npy
(1, 80, 487)
trebek_923.npy
(1, 80, 535)
trebek_781.npy
(1, 80, 451)
trebek_2120.npy
(1, 80, 662)
trebek_1572.npy
(1, 80, 336)
trebek_1786.npy
(1, 80, 764)
trebek_394.npy
(1, 80, 515)
trebek_804.npy
(1, 80, 533)
trebek_1520.npy
(1, 80, 349)
trebek_1278.npy
(1, 80, 535)
trebek_993.npy
(1, 80, 243)
trebek_583.npy
(1, 80, 614)
trebek_2527.npy
(1, 80, 489)
trebek_95.npy
(1, 80, 321)
trebek_251.npy
(1, 80, 336)
trebek_164.npy
(1, 80, 618)
trebek_1842.npy
(1, 80, 311)
trebek_2549.npy
(1, 80, 559

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [11]:
import os
import json
import hifi_gan
from hifi_gan.env import AttrDict
from hifi_gan.inference_e2e import load_checkpoint
from hifi_gan.meldataset import MAX_WAV_VALUE
from hifi_gan.models import Generator

def load_hifi_gan(path, device="cpu"):
    dev = torch.device(device)
    with open(os.path.join(path, "config.json"), "r") as f:
        config = AttrDict(json.load(f))
    generator = Generator(config).to(dev)
    for file in os.listdir(path):
        if file.startswith("g_"):
            gen_filename = file
    state_dict_g = load_checkpoint(os.path.join(path, gen_filename), dev)
    print(state_dict_g.keys())
    generator.load_state_dict(state_dict_g['generator'])
    generator.eval()
    generator.remove_weight_norm()
    return generator

In [12]:
gen = load_hifi_gan("/home/ubuntu/UNIVERSAL_V1/", device="cuda")

Loading '/home/ubuntu/UNIVERSAL_V1/g_02500000'
Complete.
dict_keys(['generator'])
Removing weight norm...


In [13]:
MAX_WAV_VALUE = 28000.0
length = output_lengths[1].item()
_mel = mel_out_postnet[1][:, :length].data
tensor_cls = torch.FloatTensor if not torch.cuda.is_available() else torch.cuda.FloatTensor
y_g_hat = gen(tensor_cls(_mel.unsqueeze(0)).to(device="cuda"))
audio = y_g_hat.squeeze()
# audio = audio * MAX_WAV_VALUE
audio = audio.unsqueeze(0)

NameError: name 'output_lengths' is not defined

In [84]:
Audio(audio.data.cpu().numpy(), rate=22050)

In [55]:
Audio("/home/ubuntu/hg_tune_data/wavs/michaelrosen9-110.wav")

In [10]:
melspec = torch.load("/home/ubuntu/michael-rosen/wavs/michaelrosen9-110.pt")

In [11]:
loaded = np.load("/home/ubuntu/hg_tune_data/mels/michaelrosen9-110.npy")

In [16]:
tensor_cls = torch.FloatTensor if not torch.cuda.is_available() else torch.cuda.FloatTensor
y_g_hat = gen(tensor_cls(loaded).to(device="cuda"))
audio = y_g_hat.squeeze()
# audio = audio * MAX_WAV_VALUE
audio = audio.unsqueeze(0)

In [18]:
Audio(audio.data.cpu().numpy(), rate=22050)

In [57]:
loaded.shape

(1, 80, 557)

In [52]:
Audio("/home/ubuntu/michael-rosen/wavs/michaelrosen7_582.wav")

In [59]:
(557 - 32) * 256

134400

In [37]:
batch[5]

tensor([296., 389.,  96., 394., 174., 163., 401., 296., 437., 228.])

In [64]:
ls ~/michael-rosen/wavs/michaelrosen9-110.pt

/home/ubuntu/michael-rosen/wavs/michaelrosen9-110.pt


In [12]:
melspec.shape

torch.Size([80, 450])

In [13]:
loaded.shape

(1, 80, 557)

In [15]:
print(np.load("/home/ubuntu/hg_tune_data/mels/michaelrosen9-107.npy").shape)
print(torch.load("/home/ubuntu/michael-rosen/wavs/michaelrosen9-107.pt").shape)

(1, 80, 395)
torch.Size([80, 395])


In [12]:
tries = 0
fails = 0
for pt in os.listdir("/home/ubuntu/trebek/wavs/"):
    if not pt.endswith(".pt"):#  or pt.startswith("michaelrosen_"):
        continue
    try:
        npmel = np.load(f"/home/ubuntu/hg_tune_data/mels/{pt.replace('pt', 'npy')}")
    except Exception:
        #print(f"np version of {pt} probably doesn't exist")
        continue
    tmel = torch.load(f"/home/ubuntu/trebek/wavs/{pt}").unsqueeze(0)
    tries += 1
    if npmel.shape != tmel.shape:
        fails += 1
        print(pt)
        #print(npmel.shape)
        #print(tmel.shape)
print(f"{tries} tries, {fails} fails")

trebek_2381.pt
trebek_979.pt
trebek_1435.pt
trebek_1015.pt
trebek_861.pt
trebek_1024.pt
trebek_808.pt
trebek_533.pt
trebek_2032.pt
trebek_2075.pt
trebek_814.pt
trebek_1795.pt
trebek_2355.pt
trebek_759.pt
trebek_1919.pt
trebek_2230.pt
trebek_736.pt
trebek_137.pt
trebek_239.pt
trebek_1983.pt
trebek_1258.pt
trebek_1776.pt
trebek_1917.pt
trebek_649.pt
trebek_2153.pt
trebek_144.pt
trebek_1231.pt
trebek_1455.pt
2450 tries, 28 fails


In [ ]:
print("422, 26")